<a href="https://colab.research.google.com/github/paulacca467/pub_selci/blob/main/secoestranversais_matlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
from scipy.interpolate import griddata

# === Funções auxiliares ===

def uv2intdir(ue, vn, decl_mag=0):
    """Converte componentes u,v em intensidade (magnitude) e direção corrigida."""
    mag = np.sqrt(ue**2 + vn**2)
    direcao = (np.degrees(np.arctan2(ue, vn)) + decl_mag) % 360
    return mag, direcao


def intdir2uv(mag, direcao, decl_mag=0, ang_rot=0):
    """Converte intensidade e direção em componentes u,v, com rotação opcional."""
    ang = np.radians(direcao + decl_mag + ang_rot)
    u = mag * np.sin(ang)
    v = mag * np.cos(ang)
    return u, v


def sw_dist(lat, lon):
    """Calcula distância incremental e ângulo entre pares de coordenadas (km e graus)."""
    R = 6371.0
    lat1, lon1 = np.radians(lat[:-1]), np.radians(lon[:-1])
    lat2, lon2 = np.radians(lat[1:]), np.radians(lon[1:])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    dist = R * c
    angle = np.degrees(np.arctan2(np.sin(dlon)*np.cos(lat2),
                                  np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dlon)))
    return dist, angle


# === Caminho dos dados ===
pasta = "/content/drive/MyDrive/PUB_SELCI/cananeia/campo_dezembro_2019/dados_ADCP/ADCP/ADCP_dados_selecionados/next_base_dados_selecionados"

subdirs = [d for d in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, d)) and not d.startswith('.')]

ANGULO, ANG, vrot_MEAN = [], [], []

for nome in subdirs:
    path = os.path.join(pasta, nome)

    def load_txt(filename):
        """Carrega arquivo de texto simples em numpy array."""
        file_path = os.path.join(path, filename)
        if not os.path.exists(file_path):
            print(f"⚠ Arquivo {filename} não encontrado em {path}")
            return None
        return np.loadtxt(file_path)

    V = load_txt(f"{nome}.spd")
    D = load_txt(f"{nome}.dir")
    VE = load_txt(f"{nome}.ve")
    VN = load_txt(f"{nome}.vn")
    GPS = load_txt(f"{nome}.gps")
    BTK = load_txt(f"{nome}.btk")

    if any(x is None for x in [V, D, VE, VN, GPS, BTK]):
        continue

    # Transpor matrizes
    V, D, VE, VN = V.T, D.T, VE.T, VN.T

    # Remove células abaixo do fundo
    mask = V != 3276.7
    max_i, max_j = np.max(np.where(mask), axis=1)
    V, D, VE, VN = V[:max_i+1, :max_j+1], D[:max_i+1, :max_j+1], VE[:max_i+1, :max_j+1], VN[:max_i+1, :max_j+1]

    # Substitui 3276.7 por NaN
    for arr in [V, D, VE, VN]:
        arr[arr == 3276.7] = np.nan

    bu, la = 1, 1
    v, d, ve, vn = V[bu-1:, la-1:], D[bu-1:, la-1:], VE[bu-1:, la-1:], VN[bu-1:, la-1:]

    # Corrige declinação magnética
    decl_mag = -20.71
    mag, direcao = uv2intdir(ve, vn, decl_mag)

    # === Componentes sem correção magnética (iguais aos originais) ===
    ue = ve.copy()
    vn = vn.copy()

    # Cálculo da profundidade média (perfil de fundo)
    p = np.nanmean(np.fliplr(BTK[:, 2:5].T), axis=0)
    n = np.fliplr(BTK[:, 0:1].T).flatten()
    n = n - (np.nanmin(n) - 1)
    fundo = np.vstack([n, p])
    fundo[1, fundo[1, :] > 30] = np.nan

    LAT = np.column_stack((GPS[:, 2], GPS[:, 5]))
    LON = np.column_stack((GPS[:, 3], GPS[:, 6]))
    LAT[0, 0], LON[0, 0] = LAT[0, 1], LON[0, 1]

    lat = np.nanmean(LAT, axis=1)
    lon = np.nanmean(LON, axis=1)

    dst, phaseangle = sw_dist(lat, lon)
    DST = np.cumsum(dst)
    X = np.insert(DST, 0, 0)

    xo = np.flip(X)
    m = np.nanmax(xo)
    dista = np.vstack([n, X[la-1:]])

    totaldist_metros = round(np.nanmax(dista[1, :]) * 1000)
    L, C = v.shape
    MAX = 17  # profundidade máxima (m)
    z = np.arange(1, MAX+1)

    # converte para m/s
    ue, vn, v, mag = ue/100, vn/100, v/100, mag/100

    # ângulo médio do transecto
    dist, phaseangle = sw_dist(lat, lon)
    phaseangle2 = np.nanmean(phaseangle[3:-1])
    ang_rot = phaseangle2 + 90

    urot, vrot = intdir2uv(mag, direcao, 0, ang_rot)
    vrotmean, urotmean = np.nanmean(vrot, axis=1), np.nanmean(urot, axis=1)

    vrotnew = np.tile(vrotmean[:, None], (1, vrot.shape[1])) * ~np.isnan(vrot)
    urotnew = np.tile(urotmean[:, None], (1, urot.shape[1])) * ~np.isnan(urot)
    vrotnew[vrotnew == 0] = np.nan
    urotnew[urotnew == 0] = np.nan

    # Plot das seções
    Min, Max = -1.0, 1.0
    zplot = z[:L]
    Xplot = X[:C]

    vrot_mean = np.nanmean(np.nanmean(vrot, axis=1))
    vrot_MEAN.append(vrot_mean)

    fig, ax = plt.subplots(figsize=(8, 5))
    cf = ax.contourf(Xplot, -zplot, vrotnew, cmap="RdBu_r", levels=50)
    plt.colorbar(cf, ax=ax)
    ax.set(xlim=(0, 0.45), ylim=(-15, -1))
    ax.set_xlabel("Distância (km)")
    ax.set_ylabel("Profundidade (m)")
    ax.set_title(f"Seção vertical de corrente paralela à costa {nome}")
    plt.savefig(f"secao_corrente_paralela_{nome}.png", dpi=200)
    plt.close(fig)

    fig, ax = plt.subplots(figsize=(8, 5))
    cf = ax.contourf(Xplot, -zplot, urotnew, cmap="RdBu_r", levels=50)
    plt.colorbar(cf, ax=ax)
    ax.set(xlim=(0, 0.45), ylim=(-15, -1))
    ax.set_xlabel("Distância (km)")
    ax.set_ylabel("Profundidade (m)")
    ax.set_title(f"Seção vertical de corrente perpendicular à costa {nome}")
    plt.savefig(f"secao_corrente_perpendicular_{nome}.png", dpi=200)
    plt.close(fig)

    # Plot do trajeto GPS
    plt.figure(figsize=(5, 5))
    plt.plot(lon, lat, '-o', markersize=3)
    plt.scatter(lon[0], lat[0], color='k', label='Início')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"Trajeto do barco - {nome}")
    plt.legend()
    plt.savefig(f"trajeto_{nome}.png", dpi=200)
    plt.close()

    ANGULO.append(phaseangle2)
    ANG.append(phaseangle)

print("✅ Conversão concluída. Gráficos salvos em PNG.")


✅ Conversão concluída. Gráficos salvos em PNG.


In [4]:
!zip secoes.zip *.png

  adding: secao_corrente_paralela_C11912170902.png (deflated 20%)
  adding: secao_corrente_paralela_C121912171029.png (deflated 19%)
  adding: secao_corrente_paralela_C131912171124.png (deflated 21%)
  adding: secao_corrente_paralela_C141912171216.png (deflated 20%)
  adding: secao_corrente_paralela_C151912171524.png (deflated 20%)
  adding: secao_corrente_paralela_C161912171636.png (deflated 20%)
  adding: secao_corrente_paralela_C171912171808.png (deflated 20%)
  adding: secao_corrente_paralela_C181912171935.png (deflated 18%)
  adding: secao_corrente_paralela_C191912172020.png (deflated 19%)
  adding: secao_corrente_paralela_C191912172151.png (deflated 20%)
  adding: secao_corrente_perpendicular_C11912170902.png (deflated 20%)
  adding: secao_corrente_perpendicular_C121912171029.png (deflated 20%)
  adding: secao_corrente_perpendicular_C131912171124.png (deflated 21%)
  adding: secao_corrente_perpendicular_C141912171216.png (deflated 20%)
  adding: secao_corrente_perpendicular_C1519